In [7]:
import whisper
import datasets
import threading
import pydub
import soundfile
import pandas as pd
import numpy as np
import librosa
import os
from pydub import AudioSegment
import os
import whisper
from pydub import AudioSegment
import datasets
import pandas as pd
import concurrent.futures

In [3]:
dataset = datasets.load_dataset("CAiRE/ASCEND")

In [4]:
# Function to separate mp3 file by timestamp
set = pd.DataFrame(dataset['train'])
display(set)

,id,path,audio,transcription,duration,language,original_speaker_id,session_id,topic
0,00000,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,我刚刚开始record,1.560,mixed,1,1,persona
1,00001,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,嗯hello我的名字叫徐妍,4.160,mixed,1,1,persona
2,00002,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,嗯初次见面nice to meet you嗯,3.320,mixed,1,1,persona
3,00003,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,今天呢我非常希望能够通过这个机会去跟你make friends,5.700,mixed,1,1,persona
4,00004,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,嗯你知道就是,2.020,zh,1,1,persona
...,...,...,...,...,...,...,...,...,...
9864,09864,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,嗯,0.550,zh,8,4,technology
9865,09865,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,her intellect intelli,1.485,en,8,3,education
9866,09866,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,学术方面的话 其实国内也有很多 学术很厉害 的 嗯 professor之类的 但是可能整体的...,14.506,mixed,8,3,education
9867,09867,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,home school education can provide an environme...,8.611,en,8,3,education


In [26]:
path = set.at[9595, 'path']
print(path)

C:\Users\saleh\.cache\huggingface\datasets\downloads\extracted\f7ac516b8b52e26639eabec77d98278b293cb5a5579773126b8901eb92fb8307\waves/ses4_spk26_L24529_475.700_8.140.wav


In [ ]:
for i in range(100):
    path = set.at[i, 'path']
    

In [20]:
path = "Alia_Bhatt_Interview_For_Film_Shandaar.mp3"
audio = AudioSegment.from_mp3(path)
audio.export("full_alia.wav", format="wav")

<_io.BufferedRandom name='full_alia.wav'>

In [37]:
path = "full_alia.wav"
name = "full_alia.wav"
total_wav = AudioSegment.from_wav(path)
total_wav_file = total_wav.export(name, format="wav")
duration = len(total_wav)
model = whisper.load_model("base")

def split_audio(file, start, end):
    wav = AudioSegment.from_wav(file)
    first_half = wav[start:end]
    second_half = wav[end:]
    first_half.export("first_half_"+file, format="wav")
    second_half.export("second_half_"+file, format="wav")

def get_highest_hi_en(file):
    audio = whisper.load_audio(file)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
    sorted_probs = [{'key': key, 'value': value} for key, value in sorted_probs]
    hi_prob = probs['hi']
    en_prob = probs['en']
    if hi_prob > en_prob:
        return {'key': 'hi','value': probs['hi']}
    else:
        return {'key': 'en', 'value': probs['en']}

def get_highest_whisper_entry(file):
    audio = whisper.load_audio(file)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
    sorted_probs = [{'key': key, 'value': value} for key, value in sorted_probs]
    return sorted_probs[0]

def binary_search(file, start, end, new_file, my_dict):
    if get_highest_hi_en(new_file)['value'] > 0.6:
        my_dict[start] = get_highest_hi_en(new_file)['key']
        print('high prob of', get_highest_hi_en(new_file)['value'] ,'that its 1 language:', my_dict)
        return my_dict
    
    if end - start <= 2000:
        my_dict[start] = get_highest_hi_en(new_file)['key']
        print('clip too short, split with prob:', get_highest_hi_en(new_file)['value'], my_dict)
        return
    
    split_position = start + np.ceil((end - start) / 2)
    split_position = int(split_position)
    split_audio(file, start, split_position)
    binary_search(file, start, split_position, "first_half_"+file, my_dict)
    
    # Update split position for the second half
    split_position = int(split_position)
    split_audio(file, split_position, end)
    binary_search(file, split_position, end, "second_half_"+file, my_dict)

    return my_dict

results = []

def chunk_audio_30_sec(file):
    files = []
    wav_file = AudioSegment.from_wav(file)
    duration = len(wav_file)
    for i in range(0, duration, 30000):
        chunk = wav_file[i:i+29999]
        name = f"{int(i/30000)}.wav"
        chunk.export(name, format="wav")
        files.append(name)
    return files

def get_whisper_translation(translated_texts, audio, curr_language):
    model = whisper.load_model("base")
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    options = whisper.DecodingOptions(language=curr_language, task='translate', fp16=False)
    result = whisper.decode(model, mel, options)
    translated_texts.append(result.text)    

def translate_audio(audio_path, segments_dict):
    full_audio = AudioSegment.from_file(audio_path, format="wav")
    
    translated_texts = []
    start_time = 0  # Start of the first segment
    current_language = list(segments_dict.values())[0]  # Language of the first segment

    threads = []
    for end_time, language in segments_dict.items():
        if language != current_language:
            # Process the accumulated segment in the current language
            start_ms = start_time
            end_ms = end_time - 1  # Adjusting to include the last moment of the previous language

            audio_segment = full_audio[start_ms:end_ms]
            temp_file = "temp_segment.wav"
            audio_segment.export(temp_file, format="wav")
            
            audio = whisper.load_audio(temp_file)
            thread = threading.Thread(target=get_whisper_translation, args=(translated_texts, audio, current_language))
            threads.append(thread)
            thread.start()
            
            os.remove(temp_file)

            # Update the start time and current language
            
            start_time = end_time
            current_language = language

    # Process the final segment
    start_ms = start_time
    end_ms = len(full_audio)  # Till the end of the audio
    audio_segment = full_audio[start_ms:end_ms]
    temp_file = "temp_segment.wav"
    audio_segment.export(temp_file, format="wav")
    audio = whisper.load_audio(temp_file)
    
    thread = threading.Thread(target=get_whisper_translation, args=(translated_texts, audio, current_language))
    threads.append(thread)
    thread.start()

    os.remove(temp_file)

    for thread in threads:
        thread.join()

    full_translation = ' '.join(translated_texts)
    return full_translation

my_dict = {}

chunk_audio_30_sec(name)
binary_search("12.wav", 0, len(AudioSegment.from_wav("12.wav")), "12.wav", my_dict)
translate_audio("12.wav", my_dict)

high prob of 0.6283053159713745 that its 1 language: {0: 'hi'}
clip too short, split with prob: 0.07496034353971481 {0: 'hi', 15000: 'en'}
clip too short, split with prob: 0.4506291151046753 {0: 'hi', 15000: 'en', 16875: 'hi'}
clip too short, split with prob: 0.0813167467713356 {0: 'hi', 15000: 'en', 16875: 'hi', 18750: 'en'}
clip too short, split with prob: 0.5608551502227783 {0: 'hi', 15000: 'en', 16875: 'hi', 18750: 'en', 20625: 'hi'}
clip too short, split with prob: 0.5360254049301147 {0: 'hi', 15000: 'en', 16875: 'hi', 18750: 'en', 20625: 'hi', 22500: 'hi'}
clip too short, split with prob: 0.2866332530975342 {0: 'hi', 15000: 'en', 16875: 'hi', 18750: 'en', 20625: 'hi', 22500: 'hi', 24375: 'hi'}
clip too short, split with prob: 0.20663239061832428 {0: 'hi', 15000: 'en', 16875: 'hi', 18750: 'en', 20625: 'hi', 22500: 'hi', 24375: 'hi', 26250: 'hi'}
clip too short, split with prob: 0.3625158965587616 {0: 'hi', 15000: 'en', 16875: 'hi', 18750: 'en', 20625: 'hi', 22500: 'hi', 24375: 'hi

"I'm going to go to church. That will be the next year. I'm here to invite you because it's been a long time. I'm going to see you later. Yes. After this, Kapoor Ancels will be released after Shandar. After that, Upta will be released and after that, he will see. I am not saying anything. What do I say? You have never done anything in your life. Have you ever put your hands on your fav? Yes. Have you ever put your hands on your fav? Have you ever put your hands on your fav? Yes. After that..."